


softmax(q.k/d_k^0.5).v = z

softmax((q_2.keys)/d_k^0.5).v = z   &emsp;&emsp;..... # q = (q_2.keys)

softmax(((W_query.x2).keys)/d_k^0.5).v = z  &emsp;&emsp;..... # q_2 = (W_query.x2)

softmax(((W_query.(embedding_sentence[1])).(W_keys.embedding_sentence))/d_k^0.5).(W_value.embedding_sentence) = z &emsp;&emsp;..... # x2 = (embedding_sentence[1]),keys = (W_keys.embedding_sentence),v = (W_value.embedding_sentence)

(24x16.16x1).()


<!--  -->

In [7]:
import math 


# Define a function to calculate positional encodings
def positional_encoding(sentence_length, embedding_dimension):
    pos_encodings = torch.zeros(sentence_length, embedding_dimension)
    position = torch.arange(0, sentence_length).unsqueeze(1).float()
    div_term = torch.exp(torch.arange(0, embedding_dimension, 2).float() * -(math.log(10000.0) / embedding_dimension))
    pos_encodings[:, 0::2] = torch.sin(position * div_term)
    pos_encodings[:, 1::2] = torch.cos(position * div_term)
    return pos_encodings

# Input text
text = "life is beautifull , life is short!"

# Create a dictionary 'dc' that maps each unique word in the text to its index in sorted order
dc = {s:i for i,s in enumerate(sorted(text.split()))}

# Print the dictionary
print({s:i for i,s in enumerate(sorted(text.split()))})

# Import the torch library
import torch

# Convert the input text into a tensor of word indices using the 'dc' dictionary
text_tensor = torch.tensor([dc[s] for s in text.split()])

# Print the tensor
print(torch.tensor([dc[s] for s in text.split()]))

# Set the random seed for reproducibility
torch.manual_seed(123)

# Get the size of the text tensor along dimension 0
print(text_tensor.size(dim=0))

# Define the embedding dimension for the words
embedding_dimension = 16

# Get the size of the text tensor along dimension 0 (again)
text_dimension = text_tensor.size(dim=0)

# Create an embedding layer that maps word indices to embedding vectors
embed = torch.nn.Embedding(text_dimension, embedding_dimension)

# Embed the input text using the embedding layer and detach it from the computation graph
embedded_sentence = embed(text_tensor).detach()

# Calculate positional encodings for the embedded sentence
pos_encodings = positional_encoding(text_dimension, embedding_dimension)
print("pos",pos_encodings)
# Print the embedded sentence and its shape
print(embedded_sentence)
print(embedded_sentence.shape)

# Define dimensions for query, key, and value vectors
d_q, d_k, d_v = 24, 24, 28
# Initialize weight matrices for query, key, and value
W_query = torch.nn.Parameter(torch.rand(d_q, embedding_dimension))
W_key = torch.nn.Parameter(torch.rand(d_k, embedding_dimension))
W_value = torch.nn.Parameter(torch.rand(d_v, embedding_dimension))

# Calculate keys and values for all words in the embedded sentence
keys = W_key.matmul(embedded_sentence.T).T
values = W_value.matmul(embedded_sentence.T).T
# Print the shapes of keys and values
print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

# Get the embedded vector for the second word in the text
x_2 = embedded_sentence[1]

# Calculate the query, key, and value vectors for the second word
query_2 = W_query.matmul(x_2)
key_2 = W_key.matmul(x_2)
value_2 = W_value.matmul(x_2)

# Print the shapes of the query, key, and value vectors
print("query_2",query_2.shape)
print("key_2",key_2.shape)
print("value_2",value_2.shape)

# Calculate the dot product of the query vector for the second word and keys for the fifth word
omega_24 = query_2.dot(keys[4])
print(omega_24)

# Calculate the dot product of the query vector for the second word and all keys
omega_2 = query_2.matmul(keys.T)
print(omega_2)

# Import the F module from torch.nn.functional
import torch.nn.functional as F

# Apply softmax to the omega values, scaling by d_k^0.5
attention_weights_2 = F.softmax(omega_2 / d_k**0.5, dim=0)
print(attention_weights_2)

# Calculate the context vector for the second word by taking a weighted sum of values
context_vector_2 = attention_weights_2.matmul(values)

# Print the shape and value of the context vector
print(context_vector_2.shape)
print(context_vector_2)


{',': 0, 'beautifull': 1, 'is': 3, 'life': 5, 'short!': 6}
tensor([5, 3, 1, 0, 5, 3, 6])
7
pos tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,
          0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          1.0000e+00],
        [ 8.4147e-01,  5.4030e-01,  3.1098e-01,  9.5042e-01,  9.9833e-02,
          9.9500e-01,  3.1618e-02,  9.9950e-01,  9.9998e-03,  9.9995e-01,
          3.1623e-03,  9.9999e-01,  1.0000e-03,  1.0000e+00,  3.1623e-04,
          1.0000e+00],
        [ 9.0930e-01, -4.1615e-01,  5.9113e-01,  8.0658e-01,  1.9867e-01,
          9.8007e-01,  6.3203e-02,  9.9800e-01,  1.9999e-02,  9.9980e-01,
          6.3245e-03,  9.9998e-01,  2.0000e-03,  1.0000e+00,  6.3246e-04,
          1.0000e+00],
        [ 1.4112e-01, -9.8999e-01,  8.1265e-01,  5.8275e-01,  2.9552e-01,
          9.5534e-01,  9.4726e-02,  9.9550e-01,  2.9995e-02,  9.9955e-01,
          9.4867e-03, 

Multi-Head Attention

suppose we have 3 attention heads, so we now extend the d′×d
 dimensional weight matrices so 3×d′×d




softmax(q.k/d_k^0.5).v = z

softmax((q_2.keys)/d_k^0.5).v = z   &emsp;&emsp;..... # q = (q_2.keys)

softmax(((W_query.x2).keys)/d_k^0.5).v = z  &emsp;&emsp;..... # q_2 = (W_query.x2)

softmax(((W_query.(embedding_sentence[1])).(W_keys.embedding_sentence))/d_k^0.5).(W_value.embedding_sentence) = z &emsp;&emsp;..... # x2 = (embedding_sentence[1]),keys = (W_keys.embedding_sentence),v = (W_value.embedding_sentence)

(3x24x16.16x1).()

(3x24x1).()

<!--  -->

In [9]:

h = 3
multihead_W_query = torch.nn.Parameter(torch.rand(h, d_q, embedding_dimension))
multihead_W_key = torch.nn.Parameter(torch.rand(h, d_k, embedding_dimension))
multihead_W_value = torch.nn.Parameter(torch.rand(h, d_v, embedding_dimension))
print(multihead_W_query.shape)

multihead_query_2 = multihead_W_query.matmul(x_2)
print(multihead_query_2)

multihead_key_2 = multihead_W_key.matmul(x_2)
multihead_value_2 = multihead_W_value.matmul(x_2)

stacked_inputs = embedded_sentence.T.repeat(3, 1, 1)
print(stacked_inputs.shape)

multihead_keys = torch.bmm(multihead_W_key, stacked_inputs)
multihead_values = torch.bmm(multihead_W_value, stacked_inputs)
print("multihead_keys.shape:", multihead_keys.shape)
print("multihead_values.shape:", multihead_values.shape)

multihead_keys = multihead_keys.permute(0, 2, 1)
multihead_values = multihead_values.permute(0, 2, 1)
print("multihead_keys.shape:", multihead_keys.shape)
print("multihead_values.shape:", multihead_values.shape)




torch.Size([3, 24, 16])
tensor([[-0.1675,  2.0724,  3.1771,  0.5790,  0.9045,  0.9083,  2.0282,  2.5198,
          1.3771,  2.7432,  1.2577,  3.1156,  0.9190,  2.5747, -0.4926,  1.5206,
          2.5045,  1.3599,  3.1511,  3.9653,  4.8202,  1.4465,  3.1969,  2.5690],
        [ 1.4567,  2.3812,  1.4731,  0.0214,  1.8357,  1.2451,  2.1410,  3.7393,
          1.2424,  3.2784,  1.4393,  3.6012,  4.1137, -0.5055,  0.7926,  3.2467,
          3.7470,  2.1107,  2.1918,  2.3549,  0.0217,  1.4700,  2.0864,  2.6827],
        [ 3.0280,  3.6926,  3.0617,  1.8447,  1.3672,  2.5542,  0.4363, -0.7668,
          1.6658,  1.5061,  2.7586,  1.4904,  3.3547,  1.2844,  1.9734,  1.4638,
         -0.7346,  4.2401,  2.5253,  2.5976,  0.7128,  1.4504,  0.8532,  2.4270]],
       grad_fn=<UnsafeViewBackward0>)
torch.Size([3, 16, 7])
multihead_keys.shape: torch.Size([3, 24, 7])
multihead_values.shape: torch.Size([3, 28, 7])
multihead_keys.shape: torch.Size([3, 7, 24])
multihead_values.shape: torch.Size([3, 7, 28]

# Cross-Attention Layer (Decoder) 

In [11]:
torch.manual_seed(123)

d = embedded_sentence.shape[1]
print("embedded_sentence.shape:", embedded_sentence.shape)

d_q, d_k, d_v = 24, 24, 28

W_query = torch.rand(d_q, d)
W_key = torch.rand(d_k, d)
W_value = torch.rand(d_v, d)

x_2 = embedded_sentence[1]
query_2 = W_query.matmul(x_2)
print("query.shape", query_2.shape)

keys = W_key.matmul(embedded_sentence.T).T
values = W_value.matmul(embedded_sentence.T).T

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

embedded_sentence_2 = torch.rand(8, 16) # 2nd input sequence

keys = W_key.matmul(embedded_sentence_2.T).T
values = W_value.matmul(embedded_sentence_2.T).T

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

embedded_sentence.shape: torch.Size([7, 16])
query.shape torch.Size([24])
keys.shape: torch.Size([7, 24])
values.shape: torch.Size([7, 28])
keys.shape: torch.Size([8, 24])
values.shape: torch.Size([8, 28])


In [5]:
import torch
import math

# Define a function to calculate positional encodings
def positional_encoding(sentence_length, embedding_dimension):
    pos_encodings = torch.zeros(sentence_length, embedding_dimension)
    position = torch.arange(0, sentence_length).unsqueeze(1).float()
    div_term = torch.exp(torch.arange(0, embedding_dimension, 2).float() * -(math.log(10000.0) / embedding_dimension))
    pos_encodings[:, 0::2] = torch.sin(position * div_term)
    pos_encodings[:, 1::2] = torch.cos(position * div_term)
    return pos_encodings

# Define the input text
text = "life is beautiful, life is short!"

# Create a dictionary 'dc' that maps each unique word in the text to its index in sorted order
dc = {s: i for i, s in enumerate(sorted(text.split()))}

# Convert the input text into a tensor of word indices using the 'dc' dictionary
text_tensor = torch.tensor([dc[s] for s in text.split()])

# Get the size of the text tensor along dimension 0
sentence_length = text_tensor.size(dim=0)

# Define the embedding dimension for the words
embedding_dimension = 16

# Create an embedding layer that maps word indices to embedding vectors
embed = torch.nn.Embedding(len(dc), embedding_dimension)

# Embed the input text using the embedding layer and detach it from the computation graph
embedded_sentence = embed(text_tensor).detach()

# Calculate positional encodings for the embedded sentence
pos_encodings = positional_encoding(sentence_length, embedding_dimension)

# Add positional encodings to the embedded sentence
embedded_sentence_with_pos = embedded_sentence + pos_encodings

# Print the embedded sentence with positional encodings and its shape
print(embedded_sentence_with_pos)
print(embedded_sentence_with_pos.shape)


IndexError: index out of range in self